# 한국어 전처리


> 솔트룩스 AI Labs NLP파트 김성현 (bananaband657@gmail.com)



## 0. Introduction

한국어에서의 다양한 전처리 방식들을 실습합니다.

* Basic
 - 가장 기초적인 전처리
 - html tag 제거
 - 숫자 제거
 - Lowercasing
 - "@%*=()/+ 와 같은 punctuation 제거
* Spell check
 - 사전 기반의 오탈자 교정
 - 줄임말 원형 복원 (e.g. I'm not happy -> I am not happy)
* Part-of-Speech
 - 형태소 분석
 - Noun, Adjective, Verb, Adverb만 학습에 사용
* Stemming
 - 형태소 분석 이후 동사 원형 복원
* Stopwords
 - 불용어 제거
* Negation
 - [논문](https://dl.acm.org/doi/pdf/10.5555/2392701.2392703)
 - 부정 표현에 대한 단순화 (e.g. I'm not happy -> I'm sad)
 - 한국어에서의 적용이 어려워, 추후 추가할 예정


먼저 실습을 위해 한국어 wikipedia 문서를 다운받도록 하겠습니다.

In [1]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1EcJpRTEdGVaYhbLE1otE5iCifj_kW1_4" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1EcJpRTEdGVaYhbLE1otE5iCifj_kW1_4" -o wiki_20190620.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1915      0 --:--:-- --:--:-- --:--:--  1915
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  476M    0  476M    0     0  67.8M      0 --:--:--  0:00:07 --:--:-- 79.5M


## 1. Basic Preprocessing

In [2]:
# 한국어 위키 데이터 load
data = open('/content/wiki_20190620.txt', 'r', encoding='utf-8')
lines = data.readlines()

In [3]:
for i in range(0, 10):
    print(lines[i])

제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

조지아 공과대학교를 졸업하였다.

그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.

1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.

그의 별명이 "땅콩 농부" 로 알려졌다.

1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.

대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.

조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.



한국어 문장 분리 라이브러리 중, 가장 성능이 좋은 tokenizer 중 하나인 kss를 설치합니다.

In [4]:
!pip install kss

     |████████████████████████████████| 42.3 MB 79 kB/s 
     |████████████████████████████████| 184 kB 67.9 MB/s 
  Created wheel for kss: filename=kss-3.1.0.4-py3-none-any.whl size=42336591 sha256=ddcbf7bd8ca8d5749775aba1389356336adca3750eb726c9344c35887f1cb60f
  Stored in directory: /root/.cache/pip/wheels/94/d8/3c/b5f02f814e08c3e2f35e32ae2ac92a34c8412ed6f92ff470ce
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=41b98522a158f4875abc37e92067b3bdc27ce6fcfad01182c1d149889e973922
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built kss emoji


In [5]:
import kss

sentence_tokenized_text = []
for i, line in enumerate(lines):
    if i > 100:     # 전체 wikipedia 문서는 사이즈가 크므로, 일부만 테스트.
        break
    line = line.strip()
    for sent in kss.split_sentences(line):
        sentence_tokenized_text.append(sent.strip())

[Korean Sentence Splitter]: Initializing Kss...


이제 `sentence_tokenized_text`에 문장 단위로 분리된 corpus가 저장되었습니다.

In [6]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [7]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [8]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

In [9]:
cleaned_corpus = []
for sent in sentence_tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [10]:
for i in range(0, 10):
    print(cleaned_corpus[i])

제임스 얼   "  지미  "   카터 주니어는 민주당 출신 미국 39번째 대통령 이다 .
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다 .
조지아 공과대학교를 졸업하였다 .
그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다 .
1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다 .
그의 별명이   "  땅콩 농부  "   로 알려졌다 .
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고 ,  1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다 .
대통령이 되기 전 조지아주 상원의원을 두번 연임했으며 ,  1971년부터 1975년까지 조지아 지사로 근무했다 .
조지아 주지사로 지내면서 ,  미국에 사는 흑인 등용법을 내세웠다 .
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워 ,  포드를 누르고 당선되었다 .


In [11]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [12]:
basic_preprocessed_corpus = clean_text(cleaned_corpus)

In [13]:
for i in range(0, 10):
    print(basic_preprocessed_corpus[i])

제임스 얼 " 지미 " 카터 주니어는 민주당 출신 미국 번째 대통령 이다 .
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다 .
조지아 공과대학교를 졸업하였다 .
그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다 .
년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다 .
그의 별명이 " 땅콩 농부 " 로 알려졌다 .
년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고 , 년 조지아 주 지사 선거에 낙선하지만 년 조지아 주 지사를 역임했다 .
대통령이 되기 전 조지아주 상원의원을 두번 연임했으며 , 년부터 년까지 조지아 지사로 근무했다 .
조지아 주지사로 지내면서 , 미국에 사는 흑인 등용법을 내세웠다 .
년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워 , 포드를 누르고 당선되었다 .


## 2. Spell check

띄어쓰기 검사로는 [한국어 띄어쓰기 검사 라이브러리](https://github.com/haven-jeon/PyKoSpacing)를 사용하고,   
맞춤법 검사로는 [한국어 맞춤법 검사 라이브러리](https://github.com/ssut/py-hanspell)와, [논문](https://link.springer.com/chapter/10.1007/978-3-030-12385-7_3)에서 사용되었던 외래어 사전을 사용하겠습니다.   
반복되는 이모티콘이나 자소는 이 [라이브러리](https://github.com/lovit/soynlp)를 이용해 필터링 하겠습니다.   

먼저 띄어쓰기 검사기를 설치하겠습니다.

In [14]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-lv24m748
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-lv24m748
     |████████████████████████████████| 454.4 MB 9.7 kB/s 
     |████████████████████████████████| 462 kB 73.9 MB/s 
     |████████████████████████████████| 4.0 MB 36.4 MB/s 
     |████████████████████████████████| 1.2 MB 67.1 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2255829 sha256=24ac72849ddcf21216efc219f3d84af109b3a61baffeb075f41491bd2ca486f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-9stw3pcm/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.

In [15]:
from pykospacing import Spacing
spacing=Spacing()
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")

"김형호 영화시장 분석가는 '1987'의 네이버 영화 정보 네티즌 10점 평에서 언급된 단어들을 지난해 12월 27일부터 올해 1월 10일까지 통계 프로그램 R과 KoNLP 패키지로 텍스트마이닝하여 분석했다."

다음으로 맞춤법 검사기를 설치하겠습니다.

In [16]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-i3qzgq_p
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-i3qzgq_p
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=1f8db85a4b264e9d2b43ffad16669d2675e44d34b62974455e2eb371a6592a6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qeqpe9xw/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [17]:
from hanspell import spell_checker
 
sent = "대체 왜 않돼는지 설명을 해바"
spelled_sent = spell_checker.check(sent)
checked_sent = spelled_sent.checked
 
print(checked_sent)

대체 왜 안되는지 설명을 해봐


다음으로는 데이터에서 반복되는 이모티콘이나 자모를 normalization을 위한 라이브러리를 설치하도록 하겠습니다.

In [18]:
!pip install soynlp

     |████████████████████████████████| 416 kB 13.1 MB/s 


In [19]:
from soynlp.normalizer import *
print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))

와하하핫


마지막으로 외래어 사전을 다운로드 받겠습니다.

In [20]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1451      0 --:--:-- --:--:-- --:--:--  1457
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 19779  100 19779    0     0  27319      0 --:--:-- --:--:-- --:--:--  170k


In [21]:
lownword_map = {}
lownword_data = open('/content/confused_loanwords.txt', 'r', encoding='utf-8')

lines = lownword_data.readlines()

for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]
    ori_word = line.split('\t')[1]
    lownword_map[miss_spell] = ori_word

In [22]:
def spell_check_text(texts):
    corpus = []
    for sent in texts:
        spaced_text = spacing(sent)
        spelled_sent = spell_checker.check(sent)
        checked_sent = spelled_sent.checked
        normalized_sent = repeat_normalize(checked_sent)
        for lownword in lownword_map:
            normalized_sent = normalized_sent.replace(lownword, lownword_map[lownword])
        corpus.append(normalized_sent)
    return corpus

In [23]:
spell_preprocessed_corpus = spell_check_text(basic_preprocessed_corpus)

## 3. Part-of-Speech 

Python 기반의 형태소 분석기 중, 성능이 가장 좋은 것 중 하나인 카카오의 [Khaiii](https://github.com/kakao/khaiii)를 사용하겠습니다.

In [24]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 1016, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1016 (delta 46), reused 76 (delta 29), pack-reused 877
Receiving objects: 100% (1016/1016), 33.06 MiB | 7.35 MiB/s, done.
Resolving deltas: 100% (404/404), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ --

In [25]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

test_sents = ["나도 모르게 사버렸다.", "행복해야해!", "내가 안 그랬어!", "나는 사지 않았어.", "하나도 안 기쁘다.", "상관하지마", "그것 좀 가져와"]

for sent in test_sents:
    for word in api.analyze(sent):
        for morph in word.morphs:
            print(morph.lex + '/' + morph.tag)
    print('\n')


나/NP
도/JX
모르/VV
게/EC
사/VV
아/EC
버리/VX
었/EP
다/EF
./SF


행복/NNG
하/XSA
아/EF
야/EC
하/VX
아/EF
!/SF


내/NP
가/JKS
안/MAG
그/VV
렇/VA
었/EP
어/EF
!/SF


나/NP
는/JX
사/VV
지/EC
않/VX
았/EP
어/EF
./SF


하나/NR
도/JX
안/MAG
기쁘/VA
다/EF
./SF


상관/NNG
하/XSV
지마/NNG


그것/NP
좀/MAG
가져오/VV
아/EC




In [26]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus


In [27]:
pos_tagged_corpus = pos_text(spell_preprocessed_corpus)

In [28]:
for i in range(0, 30):
    print(pos_tagged_corpus[i])

제임스/NNP 얼/NNG 지/NNP 미/NNG 카터/NNP 주니/NNG 어/NNP 민주당/NNP 출신/NNG 미국/NNP 번/NNB 대통령/NNG
지미/NNP 카터/NNP 조지아주/NNP 섬터/NNG 카운/NNG 티/NNP 플레인스/NNG 마을/NNG 태어나/VV
조지아/NNP 공과/NNG 대학교/NNG 졸업/NNG 하/XSV
후/NNG 해군/NNG 들어가/VV 전함/NNG 원자력/NNG 잠수/NNG 하/XSA 승무원/NNG 일/NNG 하/XSV
년/NNB 미국/NNP 해군/NNG 대위/NNG 예편/NNG 하/XSA 이후/NNG 땅콩/NNG 면화/NNG 등/NNB 가꾸/VV 많/VA 돈/NNG 벌/VV
별명/NNG 땅콩/NNG 농부/NNG 알리/VV 지/VX
년/NNB 조지아/NNP 주/NNP 상원/NNG 의원/NNG 선거/NNG 낙선/NNG 하/XSV 선거/NNG 부정/NNG 선거/NNG 입증/NNG 하/XSV 되/VV 당선/NNG 되/XSV 년/NNB 조지아/NNP 주/NNG 지사/NNG 선거/NNG 낙선/NNG 하/XSV 년/NNB 조지아/NNP 주/NNG 지사/NNG 역임/NNG 하/XSV
대통령/NNG 되/VV 전/NNG 조지아주/NNP 상원/NNG 의원/NNG 번/NNB 연임/NNG 하/XSV 년/NNB 년/NNB 조지아/NNP 지사/NNG 근무/NNG 하/XSV
조지아/NNP 주지사/NNG 지내/VV 미국/NNP 살/VV 흑인/NNG 등용/NNG 법/NNG 내세우/VV
년/NNB 대통령/NNG 선거/NNG 민주당/NNP 후보/NNG 출마/NNG 하/XSV 도덕주의/NNG 정책/NNG 내세우/VV 포/NNP 드/NNG 누르/VV 당선/NNG 되/XSV
카터/NNP 대통령/NNG 에너지/NNG 개발/NNG 촉구/NNG 하/XSV 공화당/NNP 반대/NNG 무산/NNG 되/XSV
카터/NNP 이집트/NNP 이스라엘/NNP 조정/NNG 하/XSV 캠프/NNG 데이비드/NNP 안와르/NNP 사다/NNP 트/NNG 대통령/NNG 메나헴/NNP 베기/VV 수상

## 4. Stemming

동사를 원형으로 복원하도록 하겠습니다.
규칙은 다음과 같습니다.

1. NNG|NNP|NNB + XSV|XSA --> NNG|NNP|NNB + XSV|XSA + 다
2. NNG|NNP|NNB + XSA + VX --> NNG|NNP + XSA + 다
3. VV --> VV + 다
4. VX --> VX + 다

In [29]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [30]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [31]:
stemming_corpus = stemming_text(pos_tagged_corpus)

In [32]:
for i in range(0, 30):
    print(stemming_corpus[i])

제임스/NNP 얼/NNG 지/NNP 미/NNG 카터/NNP 주니/NNG 어/NNP 민주당/NNP 출신/NNG 미국/NNP 번/NNB 대통령/NNG
지미/NNP 카터/NNP 조지아주/NNP 섬터/NNG 카운/NNG 티/NNP 플레인스/NNG 마을/NNG 태어나다/VV
조지아/NNP 공과/NNG 대학교/NNG 졸업하다/VV
후/NNG 해군/NNG 들어가다/VV 전함/NNG 원자력/NNG 잠수하다/VV 승무원/NNG 일하다/VV
년/NNB 미국/NNP 해군/NNG 대위/NNG 예편하다/VV 이후/NNG 땅콩/NNG 면화/NNG 등/NNB 가꾸다/VV 많/VA 돈/NNG 벌다/VV
별명/NNG 땅콩/NNG 농부/NNG 알리다/VV 지다/VV
년/NNB 조지아/NNP 주/NNP 상원/NNG 의원/NNG 선거/NNG 낙선하다/VV 선거/NNG 부정/NNG 선거/NNG 입증하다/VV 되다/VV 당선되다/VV 년/NNB 조지아/NNP 주/NNG 지사/NNG 선거/NNG 낙선하다/VV 년/NNB 조지아/NNP 주/NNG 지사/NNG 역임하다/VV
대통령/NNG 되다/VV 전/NNG 조지아주/NNP 상원/NNG 의원/NNG 번/NNB 연임하다/VV 년/NNB 년/NNB 조지아/NNP 지사/NNG 근무하다/VV
조지아/NNP 주지사/NNG 지내다/VV 미국/NNP 살다/VV 흑인/NNG 등용/NNG 법/NNG 내세우다/VV
년/NNB 대통령/NNG 선거/NNG 민주당/NNP 후보/NNG 출마하다/VV 도덕주의/NNG 정책/NNG 내세우다/VV 포/NNP 드/NNG 누르다/VV 당선되다/VV
카터/NNP 대통령/NNG 에너지/NNG 개발/NNG 촉구하다/VV 공화당/NNP 반대/NNG 무산되다/VV
카터/NNP 이집트/NNP 이스라엘/NNP 조정하다/VV 캠프/NNG 데이비드/NNP 안와르/NNP 사다/NNP 트/NNG 대통령/NNG 메나헴/NNP 베기다/VV 수상/NNG 함께/MAG 중동/NNP 평화/NNG 위하다/VV 캠프/NNG 데이비드/NNP 협정/NNG 체결하다/VV
그러

## 5. Stopwords

불용어는 도메인에 맞춰서 다양하게 구축될 수 있습니다.   

In [33]:
stopwords = ['데/NNB', '좀/MAG', '수/NNB', '등/NNB']

In [34]:
def remove_stopword_text(text):
    corpus = []
    for sent in text:
        modi_sent = []
        for word in sent.split(' '):
            if word not in stopwords:
                modi_sent.append(word)
        corpus.append(' '.join(modi_sent))
    return corpus

In [35]:
removed_stopword_corpus = remove_stopword_text(stemming_corpus)

In [36]:
for i in range(0, 30):
    print(removed_stopword_corpus[i])

제임스/NNP 얼/NNG 지/NNP 미/NNG 카터/NNP 주니/NNG 어/NNP 민주당/NNP 출신/NNG 미국/NNP 번/NNB 대통령/NNG
지미/NNP 카터/NNP 조지아주/NNP 섬터/NNG 카운/NNG 티/NNP 플레인스/NNG 마을/NNG 태어나다/VV
조지아/NNP 공과/NNG 대학교/NNG 졸업하다/VV
후/NNG 해군/NNG 들어가다/VV 전함/NNG 원자력/NNG 잠수하다/VV 승무원/NNG 일하다/VV
년/NNB 미국/NNP 해군/NNG 대위/NNG 예편하다/VV 이후/NNG 땅콩/NNG 면화/NNG 가꾸다/VV 많/VA 돈/NNG 벌다/VV
별명/NNG 땅콩/NNG 농부/NNG 알리다/VV 지다/VV
년/NNB 조지아/NNP 주/NNP 상원/NNG 의원/NNG 선거/NNG 낙선하다/VV 선거/NNG 부정/NNG 선거/NNG 입증하다/VV 되다/VV 당선되다/VV 년/NNB 조지아/NNP 주/NNG 지사/NNG 선거/NNG 낙선하다/VV 년/NNB 조지아/NNP 주/NNG 지사/NNG 역임하다/VV
대통령/NNG 되다/VV 전/NNG 조지아주/NNP 상원/NNG 의원/NNG 번/NNB 연임하다/VV 년/NNB 년/NNB 조지아/NNP 지사/NNG 근무하다/VV
조지아/NNP 주지사/NNG 지내다/VV 미국/NNP 살다/VV 흑인/NNG 등용/NNG 법/NNG 내세우다/VV
년/NNB 대통령/NNG 선거/NNG 민주당/NNP 후보/NNG 출마하다/VV 도덕주의/NNG 정책/NNG 내세우다/VV 포/NNP 드/NNG 누르다/VV 당선되다/VV
카터/NNP 대통령/NNG 에너지/NNG 개발/NNG 촉구하다/VV 공화당/NNP 반대/NNG 무산되다/VV
카터/NNP 이집트/NNP 이스라엘/NNP 조정하다/VV 캠프/NNG 데이비드/NNP 안와르/NNP 사다/NNP 트/NNG 대통령/NNG 메나헴/NNP 베기다/VV 수상/NNG 함께/MAG 중동/NNP 평화/NNG 위하다/VV 캠프/NNG 데이비드/NNP 협정/NNG 체결하다/VV
그러나/MAJ 